In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
train = pd.read_csv('C:\\Users\\guilh_000\Documents\\tweetsclassify\\train_tweet.csv')
test = pd.read_csv('C:\\Users\\guilh_000\\Documents\\tweetsclassify\\test_tweets.csv')

print(train.shape)
print(test.shape)

(31962, 3)
(17197, 2)


In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
train.isnull().any()
test.isnull().any()

id       False
tweet    False
dtype: bool

In [6]:


# checking out the negative comments from the train set

train[train['label'] == 0].head(10)



,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


In [7]:
# checking out the postive comments from the train set 

train[train['label'] == 1].head(10)


,id,label,tweet
13,14,1,@user #cnn calls #michigan middle school 'buil...
14,15,1,no comment! in #australia #opkillingbay #se...
17,18,1,retweet if you agree!
23,24,1,@user @user lumpy says i am a . prove it lumpy.
34,35,1,it's unbelievable that in the 21st century we'...
56,57,1,@user lets fight against #love #peace
68,69,1,ð©the white establishment can't have blk fol...
77,78,1,"@user hey, white people: you can call people '..."
82,83,1,how the #altright uses &amp; insecurity to lu...
111,112,1,@user i'm not interested in a #linguistics tha...


In [8]:
train.tail()

,id,label,tweet
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


In [9]:
from textblob import TextBlob
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J":'a',"N":'n',"V":'v',"R":'r'}
    words_and_tags = [(w, tag_dict.get(pos[0],'n'))for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag)for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

import string
import nltk
from nltk.corpus import stopwords
import re
def normalizer(tweet):
    punct = list(string.punctuation)
    stop_words = stopwords.words('english')
    additional_stop_words = ['RT','rt','via','...','http','twitpic','tinyurl','www']
    stopword_list = punct + stop_words + additional_stop_words
    
    tweet = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet)
    tweet_ = re.sub("(\w+:\/\/\S+)", " ", tweet)
    tweet__ = re.sub("[^a-zA-Z]", " ", tweet_)
    lemmatized = lemmatize_with_postag(tweet__)
    tokens = nltk.word_tokenize(lemmatized)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stopword_list, lower_case))
    return filtered_result

In [10]:
train['normalized'] = train.tweet.apply(normalizer)



In [11]:
train.tail()

,id,label,tweet,normalized
31957,31958,0,ate @user isz that youuu?ðððððð...,[youuu]
31958,31959,0,to see nina turner on the airwaves trying to...,"[nina, turner, airwave, try, wrap, mantle, gen..."
31959,31960,0,listening to sad songs on a monday morning otw...,"[sad, song, monday, morning, otw, work, sad]"
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...","[condemns, act]"
31961,31962,0,thank you @user for you follow,[follow]


In [12]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1,2))
vectorized_data =count_vectorizer.fit_transform(train['tweet'])
indexed_data =hstack((np.array(range(0,vectorized_data.shape[0]))[:,None],vectorized_data))


In [13]:
targets = train.iloc[:, 1]

from sklearn.model_selection import train_test_split 
data_train, data_test, targets_train, targets_test =train_test_split(indexed_data, targets, test_size=0.3,random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [14]:
from sklearn import svm
clf = svm.SVC(C= 1, kernel='linear')
clf.fit(data_train, targets_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
clf.score(data_test, targets_test)

0.9647512775054751

In [ ]:
from sklearn.model_selection import GridSearchCV
hiperparameters = {'kernel': ('linear','rbf','poly'),'C':[1.5, 10, 100, 1000],'gamma': [1e-7, 1e-4, 1e-2, 0.1]}
grid = GridSearchCV(svm.SVC(), hiperparameters, cv=3)
grid.fit(data_train, targets_train)

print(grid.best_params_)
print(grid.best_score_)

In [66]:
tweet_to_predict = [""]
x = count_vectorizer.transform(tweet_to_predict)
print(clf.predict(x))

[1]


In [67]:
train[train['label']==1].head(200)

,id,label,tweet,normalized
13,14,1,@user #cnn calls #michigan middle school 'buil...,"[school, build, wall, chant]"
14,15,1,no comment! in #australia #opkillingbay #se...,[]
17,18,1,retweet if you agree!,[agree]
23,24,1,@user @user lumpy says i am a . prove it lumpy.,"[prove, lumpy]"
34,35,1,it's unbelievable that in the 21st century we'...,"[unbelievable, st, century, need, something, l..."
56,57,1,@user lets fight against #love #peace,[]
68,69,1,ð©the white establishment can't have blk fol...,"[establishment, blk, folx, run, around, love, ..."
77,78,1,"@user hey, white people: you can call people '...","[people, call, people, white]"
82,83,1,how the #altright uses &amp; insecurity to lu...,"[us, amp, insecurity, lure, men]"
111,112,1,@user i'm not interested in a #linguistics tha...,"[interested, address, amp, racism, brings]"
